In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
desc_infile = '..\\dataset\\description_0.docx'
# req_infile = '..\\dataset\\requirements_0_reduced_6.docx'
req_infile = '..\\dataset\\requirements_0_reduced_6_part.docx'

In [4]:
from attr import dataclass
import docx


@dataclass
class HeadingInfo:
    text: str
    inheritance_number: int

class HeadingsTracker:
    def __init__(self):
        self.headings_stack = []
        pass

    def next_elem(self, elem) -> bool:
        HEADING_STYLE_NAME = "Heading"
        if isinstance(elem, docx.text.paragraph.Paragraph):
            style_name = elem.style.name
            if style_name.startswith(HEADING_STYLE_NAME):
                inheritance_number = int(style_name[len(HEADING_STYLE_NAME):])

                # remove lover headings from stack
                while(len(self.headings_stack) > 0 and self.headings_stack[-1].inheritance_number >= inheritance_number):
                    del self.headings_stack[-1]

                if len(elem.text) > 0: 
                    self.headings_stack.append(HeadingInfo(elem.text, inheritance_number))
                return True
        return False

    def get_text(self):
        s = ""
        for info in self.headings_stack:
            s += '#'*info.inheritance_number + ' ' + info.text + '\n'
        return s

@dataclass
class TextPartInfo:
    heading: str
    body: str
    # TODO:
    # chapter_number: int
    
def parse_docx_parts(file_path: str) -> list[TextPartInfo]:  
    document = docx.Document(file_path)
    htracker = HeadingsTracker()

    # for paragraph in document.paragraphs:
    text_parts = []
    cur_heading = ''
    cur_body = ''
    prev_is_normal = False
    for paragraph in document.iter_inner_content():
        cur_is_normal = False
        if isinstance(paragraph, docx.text.paragraph.Paragraph):
            if not htracker.next_elem(paragraph):
                if not prev_is_normal or len(paragraph.text) > 200:
                    if len(cur_body) > 0:
                        part = TextPartInfo(cur_heading, cur_body)
                        text_parts.append(part)
                    cur_heading = htracker.get_text()
                    cur_body = ''
                if len(paragraph.text) > 0 and not paragraph.text.isspace():
                    cur_body += paragraph.text + '\n'
                    cur_is_normal = True
        elif isinstance(paragraph, docx.table.Table):
            for row in paragraph.rows:
                for cell in row.cells:
                    cur_body += f"| {cell.text} "
                cur_body += "|\n"
        prev_is_normal = cur_is_normal
    
    text_parts_clear = list(filter(lambda x: len(x.body) > 0, text_parts))
    return text_parts_clear


## Step 1: Parsing

In [24]:
req_text_parts = parse_docx_parts(req_infile)
desc_text_parts = parse_docx_parts(desc_infile)

In [25]:
print(f"{len(req_text_parts)=}")
print(f"{len(desc_text_parts)=}")

len(req_text_parts)=71
len(desc_text_parts)=70


In [26]:
# FIXME: for tests
req_text_parts = req_text_parts[:5]
desc_text_parts = desc_text_parts[10:15]

## Step 2: technical requirement points finding

In [15]:
from enum import Enum
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

import request as req
import api_ollama
import api_yandex

class RequirementStatus(Enum):
    NO_INFORMATION = 0
    NOT_SATISFIED = 1
    PARTLY_SATISFIED = 2
    SATISFIED = 3

@dataclass
class RequirementPointInfo:
    text: str
    req_part_id: int
    desc_part_id: int = -1
    status: RequirementStatus = RequirementStatus.NO_INFORMATION
    # TODO:
    # req_chapter_id: int
    # desc_chapter_id: int

points = []
llm_api = api_ollama.LlamaAPIRequest(model_name="llama3")
# llm_api = api_yandex.YandexAPIRequest(model_name="yandexgpt")
for req_part_id, text_req in enumerate(req_text_parts):
    text = text_req.heading + text_req.body
    response = req.request(llm_api, text=text, role=0)
    cur_points = req.extract_points(response[0])
    
    for p in cur_points:
        points.append(RequirementPointInfo(p, req_part_id))
    
    logging.info(f"Detected points: {cur_points}")

INFO:root:Ollama request: [{'role': 'system', 'content': 'Ты профессиональный опытный инженер. Твоя задача найти в тексте ниже строгие технические требования компании-закупщика к оборудованию. Если в тексте нет технических требований, напиши `Требований нет`. Ответом должен быть список. Ответ не должен содержать юридическую информацию. Ответ должен быть максимально лаконичным и содержать только техническую информацию. Ответ должен быть на русском языке.'}, {'role': 'user', 'content': '# Общие требования\nНасосное оборудование (включая вспомогательные системы), на которое распространяется настоящий стандарт, должно конструироваться и изготавливаться в расчете на срок службы не менее 20 лет (за исключением естественно изнашиваемых деталей, согласно таблице 14) и не менее 3 лет непрерывной эксплуатации. Остановка оборудования для выполнения техобслуживания или проверки не является нарушением этого требования.\nП р и м е ч а н и е – Тяжелые условия работы, неправильная эксплуатация и ненад

In [23]:
points

[RequirementPointInfo(text='Срок службы: не менее 20 лет', req_part_id=0, desc_part_id=0, status=<RequirementStatus.PARTLY_SATISFIED: 3>),
 RequirementPointInfo(text='Срок непрерывной эксплуатации: не менее 3 лет', req_part_id=0, desc_part_id=0, status=<RequirementStatus.PARTLY_SATISFIED: 3>),
 RequirementPointInfo(text='Квалификация оборудования для выполнения техобслуживания или проверки', req_part_id=0, desc_part_id=1, status=<RequirementStatus.PARTLY_SATISFIED: 3>),
 RequirementPointInfo(text='Минимальные данные, которые должны включать:', req_part_id=0, desc_part_id=-1, status=<RequirementStatus.NO_INFORMATION: 0>),
 RequirementPointInfo(text='NPSHA', req_part_id=1, desc_part_id=-1, status=<RequirementStatus.NO_INFORMATION: 0>),
 RequirementPointInfo(text='Кривая температура/давление насыщенных паров', req_part_id=1, desc_part_id=-1, status=<RequirementStatus.NO_INFORMATION: 0>),
 RequirementPointInfo(text='Кривая температура/вязкость', req_part_id=1, desc_part_id=-1, status=<Requ

## Step 3: match requirements with description  

In [20]:
def classify_response(message) -> RequirementStatus:
    lower_message = message.lower()
    if "нет информации" in lower_message:
        reply = RequirementStatus.NO_INFORMATION
    elif lower_message.find("не соотвеству") != -1:
        reply = RequirementStatus.NOT_SATISFIED
    elif lower_message.find("частично соответству") != -1:
        reply = RequirementStatus.PARTLY_SATISFIED
    elif lower_message.find("соответству") != -1:
        reply = RequirementStatus.SATISFIED
    else:
        logging.warn(f"Message without clear answer: {message}")
        reply = RequirementStatus.NO_INFORMATION
    return reply

In [27]:
for point_id in range(len(points)):
    cur_point = points[point_id]
    for desc_part_id, desc in enumerate(desc_text_parts):
        desc_text = desc.heading + desc.body
        res = req.request(llm_api, text=desc_text, clauses=[cur_point.text], role=2)
        cur_status = classify_response(res[0])
        if cur_status.value > cur_point.status.value:
            cur_point.status = cur_status
            cur_point.desc_part_id = desc_part_id
        if cur_status == RequirementStatus.SATISFIED:
            break
    points[point_id] = cur_point

INFO:root:Ollama request: [{'role': 'system', 'content': 'Ты профессиональный опытный инженер. Ты на вход будешь получать текст с техническим описанием. Определи удовлетворяет ли текст пункту технического требования. Ответами могут быть: нет информации (о пункте ничего не сказано в тексте), несоответствует (имеется противоречивая информация), частично соответствует (нехватает информации), соответствует (информация в пункте полностью соответствует информации текста). Ответ должен быть на русском языке. Пункт технического требования: Срок службы: не менее 20 лет'}, {'role': 'user', 'content': '# Технические требования\n## Основные параметры и характеристики\nАгрегаты должны соответствовать требованиям настоящих технических условий, СТО ИНТИ S.10.5 – 2022, ТР ТС 012/2011, ТР ТС 010/2011, ГОСТ 31441.1, ГОСТ 31441.5, ГОСТ 31441.8, ГОСТ 12.2.007.0, ГОСТ 15150 и комплекту конструкторской документации.\nПоказатели назначения по перекачиваемой жидкости и условиям установки указаны в вводной час

In [28]:
points

[RequirementPointInfo(text='Срок службы: не менее 20 лет', req_part_id=0, desc_part_id=0, status=<RequirementStatus.PARTLY_SATISFIED: 3>),
 RequirementPointInfo(text='Срок непрерывной эксплуатации: не менее 3 лет', req_part_id=0, desc_part_id=0, status=<RequirementStatus.PARTLY_SATISFIED: 3>),
 RequirementPointInfo(text='Квалификация оборудования для выполнения техобслуживания или проверки', req_part_id=0, desc_part_id=1, status=<RequirementStatus.PARTLY_SATISFIED: 3>),
 RequirementPointInfo(text='Минимальные данные, которые должны включать:', req_part_id=0, desc_part_id=0, status=<RequirementStatus.SATISFIED: 1>),
 RequirementPointInfo(text='NPSHA', req_part_id=1, desc_part_id=1, status=<RequirementStatus.PARTLY_SATISFIED: 3>),
 RequirementPointInfo(text='Кривая температура/давление насыщенных паров', req_part_id=1, desc_part_id=0, status=<RequirementStatus.PARTLY_SATISFIED: 3>),
 RequirementPointInfo(text='Кривая температура/вязкость', req_part_id=1, desc_part_id=0, status=<Requireme